# Verify Source and Mart Tables are in Sync

This notebook verifies that mart tables are synchronized with source tables after running dbt with insert_overwrite.

## Purpose:
Demonstrates that Dynamic Partition Overwrite successfully refreshed only the affected dates.

## Checks:
1. Detects which dates were refreshed by comparing last_updated_at timestamps
2. Compare source vs mart counts for those dates
3. Verify last_updated_at timestamps show the refreshed partitions are newer
4. Confirm all other dates remain unchanged (older timestamps)

## Parameters:
- `catalog`: Target Unity Catalog (default: main)
- `schema`: Schema name (default: your_schema)

## Auto-detection Strategy:
Finds partitions where last_updated_at is NEWER than the baseline (earliest timestamp in the table).
This identifies the partitions that were refreshed in the most recent dbt run.

In [ ]:
# Get parameters
dbutils.widgets.text("catalog", "main", "Catalog Name")
dbutils.widgets.text("schema", "your_schema", "Schema Name")

catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")

print(f"Catalog: {catalog}")
print(f"Schema: {schema}")

# Use Delta table history to find partitions modified in the most recent operation
print(f"\n🔍 Using Delta table history to detect updated partitions...")

# Get the most recent write operation from Delta history
history_df = spark.sql(f"""
    DESCRIBE HISTORY {catalog}.{schema}.orders_mart_partitioned
    LIMIT 5
""")

print(f"  Recent operations:")
recent_operations = history_df.select("version", "timestamp", "operation", "operationParameters").collect()
for op in recent_operations[:3]:
    print(f"    Version {op['version']}: {op['operation']} at {op['timestamp']}")

# Get the most recent WRITE operation (not CREATE TABLE, etc.)
latest_write = None
for op in recent_operations:
    if op['operation'] in ['WRITE', 'MERGE', 'UPDATE']:
        latest_write = op
        break

if latest_write:
    print(f"\n  Most recent write operation:")
    print(f"    Version: {latest_write['version']}")
    print(f"    Timestamp: {latest_write['timestamp']}")
    print(f"    Operation: {latest_write['operation']}")
    
    # Extract partition predicates from operation parameters
    operation_params = latest_write['operationParameters']
    if operation_params and 'predicate' in operation_params:
        predicate = operation_params['predicate']
        print(f"    Predicate: {predicate}")
    
    # Get partitions that were modified in this version
    # Use Delta's change data to find affected partitions
    try:
        # Query the specific version to see what partitions exist
        version_data = spark.sql(f"""
            SELECT DISTINCT order_date
            FROM {catalog}.{schema}.orders_mart_partitioned
            VERSION AS OF {latest_write['version']}
            ORDER BY order_date DESC
            LIMIT 10
        """).collect()
        
        # Compare with previous version to find changed partitions
        if latest_write['version'] > 0:
            prev_version_data = spark.sql(f"""
                SELECT DISTINCT order_date, last_updated_at
                FROM {catalog}.{schema}.orders_mart_partitioned
                VERSION AS OF {latest_write['version'] - 1}
            """).collect()
            
            current_version_data = spark.sql(f"""
                SELECT DISTINCT order_date, last_updated_at
                FROM {catalog}.{schema}.orders_mart_partitioned
                VERSION AS OF {latest_write['version']}
            """).collect()
            
            # Find partitions with different timestamps
            prev_timestamps = {row['order_date']: row['last_updated_at'] for row in prev_version_data}
            curr_timestamps = {row['order_date']: row['last_updated_at'] for row in current_version_data}
            
            changed_dates = []
            for date, curr_ts in curr_timestamps.items():
                prev_ts = prev_timestamps.get(date)
                if prev_ts is None or curr_ts != prev_ts:
                    changed_dates.append(date)
            
            target_dates = [date.strftime('%Y-%m-%d') for date in sorted(changed_dates)]
            
            print(f"\n✅ Found {len(target_dates)} partitions modified in version {latest_write['version']}:")
            print(f"   {', '.join(target_dates)}")
            print(f"\n   These partitions were refreshed by the most recent dbt run (Task 4)")
        else:
            # First version - all partitions are new
            print(f"   This is the initial version - all partitions were created")
            target_dates = []
    except Exception as e:
        print(f"   Could not determine changed partitions from Delta history: {e}")
        print(f"   Falling back to timestamp-based approach...")
        
        # Fallback: Use the top 3 newest timestamps
        recently_updated_df = spark.sql(f"""
            SELECT order_date, last_updated_at
            FROM {catalog}.{schema}.orders_mart_partitioned
            ORDER BY last_updated_at DESC
            LIMIT 3
        """).collect()
        target_dates = [row['order_date'].strftime('%Y-%m-%d') for row in recently_updated_df]
        print(f"   Using 3 most recently updated partitions: {', '.join(target_dates)}")
else:
    print(f"\n⚠️  No write operations found in recent history")
    print(f"   Falling back to timestamp-based approach...")
    
    # Fallback: Use the top 3 newest timestamps
    recently_updated_df = spark.sql(f"""
        SELECT order_date, last_updated_at
        FROM {catalog}.{schema}.orders_mart_partitioned
        ORDER BY last_updated_at DESC
        LIMIT 3
    """).collect()
    target_dates = [row['order_date'].strftime('%Y-%m-%d') for row in recently_updated_df]
    print(f"   Using 3 most recently updated partitions: {', '.join(target_dates)}")

print(f"\nVerifying dates: {target_dates}")

In [ ]:
# Verify target dates are in sync
from pyspark.sql import functions as F
from datetime import datetime

print("\n" + "="*80)
print("VERIFICATION: Source vs Mart Comparison for Target Dates")
print("="*80)

all_in_sync = True

for date in target_dates:
    # Source table counts
    count_partitioned_src = spark.sql(f"""
        SELECT COUNT(*) as cnt
        FROM {catalog}.{schema}.orders_partitioned
        WHERE order_date = '{date}'
    """).collect()[0]['cnt']

    count_liquid_src = spark.sql(f"""
        SELECT COUNT(*) as cnt
        FROM {catalog}.{schema}.orders_liquid
        WHERE order_date = '{date}'
    """).collect()[0]['cnt']

    # Mart table aggregations
    mart_partitioned = spark.sql(f"""
        SELECT total_orders, last_updated_at
        FROM {catalog}.{schema}.orders_mart_partitioned
        WHERE order_date = '{date}'
    """).collect()

    mart_liquid_count = spark.sql(f"""
        SELECT 
            SUM(total_orders) as total_orders,
            MAX(last_updated_at) as last_updated_at
        FROM {catalog}.{schema}.orders_mart_liquid
        WHERE order_date = '{date}'
    """).collect()

    mart_part_orders = mart_partitioned[0]['total_orders'] if mart_partitioned else 0
    mart_part_updated = mart_partitioned[0]['last_updated_at'] if mart_partitioned else None
    mart_liq_orders = mart_liquid_count[0]['total_orders'] if mart_liquid_count and mart_liquid_count[0]['total_orders'] else 0
    mart_liq_updated = mart_liquid_count[0]['last_updated_at'] if mart_liquid_count else None

    # Check if in sync
    part_in_sync = (count_partitioned_src == mart_part_orders)
    liq_in_sync = (count_liquid_src == mart_liq_orders)
    
    if not (part_in_sync and liq_in_sync):
        all_in_sync = False

    # Calculate minutes since update
    if mart_part_updated:
        minutes_ago = (datetime.now() - mart_part_updated).total_seconds() / 60
    else:
        minutes_ago = None

    print(f"\n{date}:")
    print(f"  Source Tables:")
    print(f"    orders_partitioned: {count_partitioned_src:,} orders")
    print(f"    orders_liquid:      {count_liquid_src:,} orders")
    print(f"  Mart Tables:")
    print(f"    orders_mart_partitioned: {mart_part_orders:,} total_orders")
    print(f"    orders_mart_liquid:      {mart_liq_orders:,} total_orders")
    print(f"  Last Updated:")
    if mart_part_updated:
        print(f"    {mart_part_updated} ({minutes_ago:.1f} minutes ago)")
    else:
        print(f"    Never updated")
    
    if part_in_sync and liq_in_sync:
        print(f"  ✅ IN SYNC: Source and Mart both show {count_partitioned_src:,} orders")
    else:
        print(f"  ❌ OUT OF SYNC:")
        if not part_in_sync:
            print(f"     - Partitioned: Source has {count_partitioned_src:,}, Mart has {mart_part_orders:,}")
        if not liq_in_sync:
            print(f"     - Liquid: Source has {count_liquid_src:,}, Mart has {mart_liq_orders:,}")

In [ ]:
# Check a few other dates to confirm they were NOT updated
print("\n" + "="*80)
print("VERIFICATION: Other Dates Should NOT Be Recently Updated")
print("="*80)

# Get a sample of other dates
other_dates_df = spark.sql(f"""
    SELECT DISTINCT order_date
    FROM {catalog}.{schema}.orders_partitioned
    WHERE order_date NOT IN ({','.join(["'" + d + "'" for d in target_dates])})
    ORDER BY order_date
    LIMIT 5
""").collect()

other_dates = [row['order_date'].strftime('%Y-%m-%d') for row in other_dates_df]

print(f"\nChecking {len(other_dates)} sample dates that should NOT have been refreshed:")
print(f"Dates: {', '.join(other_dates)}\n")

for date in other_dates:
    mart_info = spark.sql(f"""
        SELECT last_updated_at
        FROM {catalog}.{schema}.orders_mart_partitioned
        WHERE order_date = '{date}'
    """).collect()
    
    if mart_info:
        last_updated = mart_info[0]['last_updated_at']
        hours_ago = (datetime.now() - last_updated).total_seconds() / 3600
        
        if hours_ago > 24:
            print(f"  {date}: ✅ Last updated {hours_ago:.1f} hours ago (not recently refreshed)")
        else:
            print(f"  {date}: ⚠️  Last updated {hours_ago:.1f} hours ago (recently refreshed - unexpected!)")
    else:
        print(f"  {date}: ❌ No data in mart table")

In [ ]:
# Display final summary
print("\n" + "="*80)
if all_in_sync:
    print("✅ SUCCESS: DYNAMIC PARTITION OVERWRITE VERIFICATION PASSED")
else:
    print("❌ FAILURE: TABLES ARE STILL OUT OF SYNC")
print("="*80)

if all_in_sync:
    # Get total partition count dynamically
    total_partitions = spark.sql(f"""
        SELECT COUNT(DISTINCT order_date) as cnt
        FROM {catalog}.{schema}.orders_mart_partitioned
    """).collect()[0]['cnt']
    
    unchanged_partitions = total_partitions - len(target_dates)
    savings_pct = 100 - (len(target_dates) / total_partitions * 100) if total_partitions > 0 else 0
    
    print(f"\n🎯 VERIFICATION COMPLETE:")
    print(f"  ✅ All {len(target_dates)} target dates are in sync")
    print(f"  ✅ Source table counts match mart table aggregations")
    print(f"  ✅ last_updated_at timestamps show recent refresh")
    print(f"  ✅ Other dates were NOT unnecessarily refreshed")
    print(f"\n💡 This proves Dynamic Partition Overwrite worked correctly:")
    print(f"  - Automatically detected which partitions had new data")
    print(f"  - Refreshed ONLY those {len(target_dates)} partitions")
    print(f"  - Left all other partitions unchanged (efficient!)")
    print(f"\n📊 Efficiency:")
    print(f"  - Refreshed: {len(target_dates)} dates")
    print(f"  - Unchanged: {unchanged_partitions} dates")
    print(f"  - Savings: {savings_pct:.1f}% less work than full refresh!")
else:
    print(f"\n❌ ISSUE DETECTED:")
    print(f"  Some target dates are still out of sync.")
    print(f"  Please check:")
    print(f"  1. Did dbt run complete successfully?")
    print(f"  2. Were the late-arriving orders inserted with recent created_at timestamp?")
    print(f"  3. Does the source data actually have data for these dates?")

print("="*80)